In [1]:
from magma import *
from loam.boards.icestick import IceStick
from mantle import *

import mantle lattice ice40
import mantle lattice mantle40


<img src="images/counter_diagram.jpg",width=500,height=500>

In [2]:
def DefineCounter(n, ce=False):

    name = "Counter{}{}".format(n, ce)

    args = ["O", Out(Bits(n)), "COUT", Out(Bit)] + ClockInterface(ce)
    Counter = DefineCircuit(name, *args)

    add = Adders(n, cin=False, cout=True)
    reg = Register(n, ce=ce)

    wire( reg.O, add.I0 )
    wire( int2seq(1, n), add.I1 )

    wire(add.O, reg.I)

    wire( reg.O, Counter.O )

    wire( add.COUT, Counter.COUT )

    wireclock(Counter, reg)

    EndCircuit()
    return Counter


In [3]:
from magma.backend.verilog import compile as compile_verilog
print(compile_verilog(DefineCounter(4)))

compiling Addcout4
compiling Register4
compiling Counter4False
module Addcout4 (input [3:0] I0, input [3:0] I1, output [3:0] O, output  COUT);
wire  inst0_O;
wire  inst1_CO;
wire  inst2_O;
wire  inst3_CO;
wire  inst4_O;
wire  inst5_CO;
wire  inst6_O;
wire  inst7_CO;
SB_LUT4 #(.LUT_INIT(16'hC33C)) inst0 (.I0(1'b0), .I1(I0[0]), .I2(I1[0]), .I3(1'b0), .O(inst0_O));
SB_CARRY inst1 (.I0(I0[0]), .I1(I1[0]), .CI(1'b0), .CO(inst1_CO));
SB_LUT4 #(.LUT_INIT(16'hC33C)) inst2 (.I0(1'b0), .I1(I0[1]), .I2(I1[1]), .I3(inst1_CO), .O(inst2_O));
SB_CARRY inst3 (.I0(I0[1]), .I1(I1[1]), .CI(inst1_CO), .CO(inst3_CO));
SB_LUT4 #(.LUT_INIT(16'hC33C)) inst4 (.I0(1'b0), .I1(I0[2]), .I2(I1[2]), .I3(inst3_CO), .O(inst4_O));
SB_CARRY inst5 (.I0(I0[2]), .I1(I1[2]), .CI(inst3_CO), .CO(inst5_CO));
SB_LUT4 #(.LUT_INIT(16'hC33C)) inst6 (.I0(1'b0), .I1(I0[3]), .I2(I1[3]), .I3(inst5_CO), .O(inst6_O));
SB_CARRY inst7 (.I0(I0[3]), .I1(I1[3]), .CI(inst5_CO), .CO(inst7_CO));
assign O = {inst6_O,inst4_O,inst2_O,inst0_O};
ass

In [4]:
icestick = IceStick()

icestick.Clock.on()
icestick.D1.on()
icestick.D2.on()
icestick.D3.on()
icestick.D4.on()
icestick.D5.on()

main = icestick.main()

counter4 = DefineCounter(5, ce=True)()
counter23 = DefineCounter(23)()
wire(counter23.COUT, counter4.CE)
wire(counter4.O, [main.D1, main.D2, main.D3, main.D4, main.D5])

In [8]:
print(compile_verilog(main))


compiling Addcout5
compiling Register5CE
compiling Counter5True
compiling Addcout23
compiling Register23
compiling Counter23False
compiling main
module Addcout5 (input [4:0] I0, input [4:0] I1, output [4:0] O, output  COUT);
wire  inst0_O;
wire  inst1_CO;
wire  inst2_O;
wire  inst3_CO;
wire  inst4_O;
wire  inst5_CO;
wire  inst6_O;
wire  inst7_CO;
wire  inst8_O;
wire  inst9_CO;
SB_LUT4 #(.LUT_INIT(16'hC33C)) inst0 (.I0(1'b0), .I1(I0[0]), .I2(I1[0]), .I3(1'b0), .O(inst0_O));
SB_CARRY inst1 (.I0(I0[0]), .I1(I1[0]), .CI(1'b0), .CO(inst1_CO));
SB_LUT4 #(.LUT_INIT(16'hC33C)) inst2 (.I0(1'b0), .I1(I0[1]), .I2(I1[1]), .I3(inst1_CO), .O(inst2_O));
SB_CARRY inst3 (.I0(I0[1]), .I1(I1[1]), .CI(inst1_CO), .CO(inst3_CO));
SB_LUT4 #(.LUT_INIT(16'hC33C)) inst4 (.I0(1'b0), .I1(I0[2]), .I2(I1[2]), .I3(inst3_CO), .O(inst4_O));
SB_CARRY inst5 (.I0(I0[2]), .I1(I1[2]), .CI(inst3_CO), .CO(inst5_CO));
SB_LUT4 #(.LUT_INIT(16'hC33C)) inst6 (.I0(1'b0), .I1(I0[3]), .I2(I1[3]), .I3(inst5_CO), .O(inst6_O));
SB_CARR

In [6]:
%%bash
cd build
yosys -q -p 'synth_ice40 -top main -blif ice_counter.blif' ice_counter.v
arachne-pnr -q -d 1k -o ice_counter.txt -p ice_counter.pcf ice_counter.blif
icepack ice_counter.txt ice_counter.bin

iceprog ice_counter.bin

init..
cdone: high
reset..
cdone: low
flash ID: 0x20 0xBA 0x16 0x10 0x00 0x00 0x23 0x51 0x73 0x10 0x23 0x00 0x35 0x00 0x35 0x06 0x06 0x15 0x43 0xB6
file size: 32220
erase 64kB sector at 0x000000..
programming..
reading..
VERIFY OK
cdone: high
Bye.
